In [ ]:
import re
import os
import gc
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
import transformers
from transformers import RobertaTokenizer, TFRobertaModel
from kaggle_datasets import KaggleDatasets
tf.get_logger().setLevel(logging.ERROR)
from kaggle_datasets import KaggleDatasets
from cloud_tpu_client import Client
from sklearn import datasets
from sklearn import model_selection

from numba import cuda



from IPython.display import SVG


from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger


import tensorflow_addons as tfa

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

import matplotlib.pyplot as plt

from transformers import AutoTokenizer,AutoModelForSeq2SeqLM,AutoModel

![talkIsCheap](https://quotefancy.com/media/wallpaper/3840x2160/1700728-Linus-Torvalds-Quote-Talk-is-cheap-Show-me-the-code.jpg)

In [ ]:
#Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')

In [ ]:
tf.__version__

In [ ]:
transformers.__version__

for training on tpu, use tpu_on = 1. i tried hard on tpu but couldn't get better result after tpu training,compared to gpu training

In [ ]:
# Configurations
EPOCHS = 100

tpu_on = 0
debug = 1
# Seed
SEED = 12345

# Learning rate
LR = 1e-4
LR1 = 1e-3

# Verbosity
VERBOSE = 2


# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)



In [ ]:
if(not debug):
    print("saving tokenizer,model for offline inference...\n")
    save_path = '/kaggle/working/'
    tokenizer.save_pretrained(save_path)
    def build_roberta_base_model():
        transformer = TFRobertaModel.from_pretrained(MODEL)

        return transformer


    modelweight = build_roberta_base_model()
    modelweight.save_weights('tf_model.h5')

    configuration = modelweight.config

    configuration.save_pretrained(save_path)

    del modelweight
    if(tpu_on):
        print("\ntpu on, clearing memory for next fold........\n\n\n")
        tf.tpu.experimental.initialize_tpu_system(tpu) 
    gc.collect()

In [ ]:
tr = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
tr.excerpt[118]

In [ ]:
max = 0
for i in range(len(tr)):
    if(len(tr.excerpt[i]) > max):
        max = len(tr.excerpt[i])
    
max

In [ ]:
# https://www.kaggle.com/abhishek/step-1-create-folds/notebook

def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["target"], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

In [ ]:
#LR = 0.000040
def build_lrfn(lr_start=0.000410, lr_max=0.00001, 
               lr_min=0.00001, lr_rampup_epochs=7, 
               lr_sustain_epochs=0, lr_exp_decay=.87):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

plt.figure(figsize=(10, 7))

lrfn = build_lrfn()
plt.plot([i for i in range(35)], [lrfn(i) for i in range(35)]);

In [ ]:
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
import numpy as np

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
from transformers import TFRobertaForSequenceClassification
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# This function tokenize the text according to a transformers model tokenizer
def regular_encode(texts, tokenizer = None, maxlen = 512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = maxlen,
    )
    
    return np.array(enc_di['input_ids'])

# This function encode our training sentences
def encode_texts(x_train,tokenizer, x_val, MAX_LEN):
    x_train = regular_encode(x_train.tolist(), tokenizer = tokenizer, maxlen = MAX_LEN)
    x_val = regular_encode(x_val.tolist(), tokenizer = tokenizer, maxlen = MAX_LEN)
    return x_train, x_val

# Function to transform arrays to tensors
def transform_to_tensors(x_train, x_val, y_train, y_val,BATCH_SIZE):
    
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_train, y_train))
        .repeat()
        .shuffle(2048)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    
    valid_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_val, y_val))
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    
    return train_dataset, valid_dataset

# Function to build our model
def build_roberta_model(max_len = 512,MODEL = None):
    #transformer = TFRobertaModel.from_pretrained('../input/training-tf-roberta-large-on-mlm/mlm_tf-roberta-large/')
    transformer = TFRobertaModel.from_pretrained(MODEL)
    #transformer = AutoModel.from_pretrained(MODEL)

    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
  
    # We only need the cls_token, resulting in a 2d array
    cls_token = sequence_output[:, 0, :]
    #cls_token = tf.keras.layers.Dropout(0.3)(cls_token)

    output = tf.keras.layers.Dense(1, activation = 'linear', dtype = 'float32')(cls_token)
    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = [output])
    ''' 
    cycle = tfa.optimizers.CyclicalLearningRate(
            initial_learning_rate = LR,
            maximal_learning_rate =  1e-3,
            step_size =  200,
            scale_fn = lambda x: 1.,
            scale_mode = 'cycle',
            name = 'CyclicalLearningRate'
    )
    '''
    optimizer = tf.keras.optimizers.Adam(lr = LR)
    opt = tfa.optimizers.SWA(optimizer, start_averaging=8, average_period=3)
    model.compile(optimizer = opt ,  #tfa.optimizers.RectifiedAdam(lr = LR)
                  loss = [tf.keras.losses.MeanSquaredError()],
                  metrics = [tf.keras.metrics.RootMeanSquaredError()])
    return model


In [ ]:
os.listdir('../')

In [ ]:
# Function to build our model
from transformers import RobertaConfig
def build_model(max_len = 512,MODEL = None):
    print("extract embedding... ")
    ''' 
    comment
    '''
    config = RobertaConfig(dropout=0.4, attention_dropout=0.4)
    config.output_hidden_states = False
    #transformer_model = TFRobertaModel.from_pretrained(MODEL, config = config)
    
    transformer_model = TFRobertaModel.from_pretrained('../input/distilroberta-base-pretrained-on-commonlit/mlm_tfdistilroberta-base/', config = config)

    #transformer_model = TFRobertaModel.from_pretrained(MODEL)

    #input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
    #input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 
    #embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]

    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    embedding_layer = transformer_model(input_word_ids)[0]
    
    
    cls_token = embedding_layer[:,0,:]
    X =  tfa.layers.GroupNormalization()(cls_token)
    
 
    X = tf.keras.layers.Dense(4096, activation='relu')(X)
   
   
    X = tf.keras.layers.Dense(1, activation='linear')(X)
  

    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = [X])
    ''' 
    for layer in model.layers[:2]:
        layer.trainable = False
    '''
    optimizer = tf.keras.optimizers.Adam(lr = LR)
    optimizer = tfa.optimizers.SWA(optimizer, start_averaging=30, average_period=3)
    model.compile(optimizer = optimizer ,  #tfa.optimizers.RectifiedAdam(lr = LR)
                  loss = [tf.keras.losses.MeanSquaredError()],
                  metrics = [tf.keras.metrics.RootMeanSquaredError()])
    return model


In [ ]:

# Function to train and evaluate our model
def train_and_evaluate(trainStrategy = "stratifiedkfold",modelType = "xlm"):
    
    # Read our training data
    #df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
    
    # read training data
    df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
    # Create out of folds array to store predictions
    oof_predictions = np.zeros(len(df))
    
    print(trainStrategy,modelType)
    
    if(modelType == "distilroberta"):
        print("distilroberta base model training........")
        # Number of folds for training
        FOLDS = 5
        # Max length
        MAX_LEN = 300
        # Get the trained model we want to use
        MODEL = 'distilroberta-base' #jplu/tf-xlm-roberta-base

        # Let's load our model tokenizer
        tokenizer = RobertaTokenizer.from_pretrained('../input/distilroberta-base-pretrained-on-commonlit/mlm_tfdistilroberta-base/')
        #tokenizer = AutoTokenizer.from_pretrained(MODEL)
      
        BATCH_SIZE = 32 * strategy.num_replicas_in_sync 
        
        # Learning rate
        LR = 1e-4
        LR1 = 1e-3


    else:
        print("xlmr base model training........")
        # Number of folds for training
        FOLDS = 5
        # Max length
        MAX_LEN = 192
        # Get the trained model we want to use
        MODEL = 'jplu/tf-xlm-roberta-base' #jplu/tf-xlm-roberta-large google/mt5-large
        # Let's load our model tokenizer
        #tokenizer = RobertaTokenizer.from_pretrained(MODEL)
        tokenizer = AutoTokenizer.from_pretrained(MODEL)
        BATCH_SIZE = 16 * strategy.num_replicas_in_sync 
        
       

    print(MAX_LEN)
        

    #df = clean_data(df, input_columns ) 
    if (trainStrategy == "stratifiedkfold"):
        
        # create folds
        df = create_folds(df, num_splits=FOLDS)


        # Seed everything
        seed_everything(SEED)

        # Initiate kfold object with shuffle and a specific seed
        #kfold = KFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
        
        for fold in range(FOLDS):
         
            trn_ind = np.where((df['kfold'] != fold))[0]
            val_ind = np.where((df['kfold'] == fold))[0]

            print(len(trn_ind),'=total training indexes \n \n',len(val_ind),'=total validation indexes \n')
            print("........................\n")
            #print(val_ind)
            print(f'Training fold {fold + 1}')
            K.clear_session()
            # Get text features and target
            x_train, x_val = df['excerpt'].iloc[trn_ind], df['excerpt'].iloc[val_ind]
            y_train, y_val = df['target'].iloc[trn_ind].values, df['target'].iloc[val_ind].values
            # Encode our text with Roberta tokenizer
            x_train, x_val = encode_texts(x_train,tokenizer, x_val, MAX_LEN)
            # Function to transform our numpy array to a tf Dataset
            train_dataset, valid_dataset = transform_to_tensors(x_train, x_val, y_train, y_val,BATCH_SIZE)
            # Build model
            with strategy.scope():
                #model = build_roberta_model(max_len = MAX_LEN,MODEL = MODEL)
                model =  build_model(max_len = MAX_LEN,MODEL = MODEL)
            # Model checkpoint
            ''' 
            count = 0
            for layer in model.layers:
                count +=1
                if count < 5: #freezing first 4 layers
                    print("freezing layers...")
                    layer.trainable = False
            
            for w in model.get_layer(MODEL).weights:
                print("freezing .......")
                w._trainable = False
            '''
            
            '''
            for layer in model.layer[:2]:
                layer.trainable = False
            '''
            checkpoint = tf.keras.callbacks.ModelCheckpoint(f'Roberta_{modelType}_{trainStrategy}_{SEED}_{fold + 1}.h5', 
                                                            monitor = 'val_root_mean_squared_error', 
                                                            verbose = VERBOSE, 
                                                            save_best_only = True,
                                                            save_weights_only = True, 
                                                            mode = 'min')
      
            es = EarlyStopping(monitor='val_root_mean_squared_error', mode='min', patience=20, 
                       restore_best_weights=True, verbose=1)
            lr_callback = LearningRateScheduler(lrfn, verbose=1)

            #LR = 0.000040

            reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                        factor=0.15, patience=15, 
                                        verbose=1, mode='min', 
                                        epsilon=0.0001, cooldown=1, min_lr=0.00001)


            steps = x_train.shape[0] // BATCH_SIZE 
            print("total steps",steps)
            # Set CLR options
            clr_step_size = steps
            base_lr = 1e-4
            max_lr = 1e-3
            mode='triangular'
            clr = CyclicLR(base_lr=base_lr, max_lr=max_lr, step_size=clr_step_size, mode=mode)


            # Training phase
            callback_list = [checkpoint,clr] #lr_callback reduceLROnPlat es
            #model.load_weights(f'Roberta_{modelType}_{trainStrategy}_{SEED}_{fold + 1}.h5')
            history = model.fit(train_dataset,
                                batch_size = BATCH_SIZE,
                                epochs = EPOCHS,
                                verbose = VERBOSE,
                                callbacks = [callback_list],
                                validation_data = valid_dataset,
                                steps_per_epoch = steps)
            
            # Load best epoch weights
            model.load_weights(f'Roberta_{modelType}_{trainStrategy}_{SEED}_{fold + 1}.h5')
            # Predict validation set to save them in the out of folds array
            val_pred = model.predict(valid_dataset)
            oof_predictions[val_ind] = val_pred.reshape(-1)
            print("\n\n\n\n")
            print('-'*50)
            del train_dataset, valid_dataset
            del model
            if(tpu_on):
                print("\ntpu on, clearing memory for next fold........\n\n\n")
                tf.tpu.experimental.initialize_tpu_system(tpu) 
            gc.collect()
         

    else:
        print(trainStrategy,'\n')
        # Initiate kfold object with shuffle and a specific seed
        kfold = KFold(n_splits = FOLDS, shuffle = True, random_state = SEED)


        for fold, (trn_ind, val_ind) in enumerate(kfold.split(df)):

            print('-'*50)
            print(f'Training fold {fold + 1}')
            K.clear_session()
            # Get text features and target
            x_train, x_val = df['excerpt'].iloc[trn_ind], df['excerpt'].iloc[val_ind]
            y_train, y_val = df['target'].iloc[trn_ind].values, df['target'].iloc[val_ind].values
            # Encode our text with Roberta tokenizer
            x_train, x_val = encode_texts(x_train,tokenizer, x_val, MAX_LEN)
            # Function to transform our numpy array to a tf Dataset
            train_dataset, valid_dataset = transform_to_tensors(x_train, x_val, y_train, y_val,BATCH_SIZE)
            # Build model
            
            with strategy.scope():
                model = build_roberta_model(max_len = MAX_LEN,MODEL = MODEL)
            # Model checkpoint
            checkpoint = tf.keras.callbacks.ModelCheckpoint(f'Roberta_{modelType}_{trainStrategy}_{SEED}_{fold + 1}.h5', 
                                                            monitor = 'val_root_mean_squared_error', 
                                                            verbose = VERBOSE, 
                                                            save_best_only = True,
                                                            save_weights_only = True, 
                                                            mode = 'min')
            steps = x_train.shape[0] // BATCH_SIZE 
            # Set CLR options
            clr_step_size = steps
            base_lr = LR
            max_lr = LR1
            mode='triangular'
            clr = CyclicLR(base_lr=base_lr, max_lr=max_lr, step_size=clr_step_size, mode=mode)
            
            es = EarlyStopping(monitor='val_root_mean_squared_error', mode='min', patience=20, 
                       restore_best_weights=True, verbose=1)
            
            reduceLROnPlat = ReduceLROnPlateau(monitor='val_root_mean_squared_error',  
                                        factor=0.07, patience=10, 
                                        verbose=1, mode='min', 
                                        epsilon=0.0001, cooldown=1, min_lr=0.00001)


            # Training phase
            callback_list = [checkpoint,reduceLROnPlat] #lr_callback reduceLROnPlat
            #model.load_weights('../input/commonlit-readability-roberta-tf/Roberta_Base_123_1.h5')
         
            # Training phase
            history = model.fit(train_dataset,
                                batch_size = BATCH_SIZE,
                                epochs = EPOCHS,
                                verbose = VERBOSE,
                                callbacks = [callback_list],
                                validation_data = valid_dataset,
                                steps_per_epoch = steps)


            # Load best epoch weights
            model.load_weights(f'Roberta_{modelType}_{trainStrategy}_{SEED}_{fold + 1}.h5')
            # Predict validation set to save them in the out of folds array
            val_pred = model.predict(valid_dataset)
            oof_predictions[val_ind] = val_pred.reshape(-1)
            print("\n\n\n\n")
            print('-'*50)
            del train_dataset, valid_dataset
            del model
            if(tpu_on):
                print("\ntpu on, clearing memory for next fold........\n\n\n")
                tf.tpu.experimental.initialize_tpu_system(tpu) 
            gc.collect()
         


    

    print('\n')
    print('-'*50)
    # Calculate out of folds root mean squared error
    oof_rmse = np.sqrt(mean_squared_error(df['target'], oof_predictions))
    print(f'Our out of folds RMSE is {oof_rmse}')
    

**Experiment - 1**

train distilroberta base 5 stratifiedkfold 

In [ ]:
%%time

train_and_evaluate(trainStrategy="kfold",modelType = "distilroberta")#stratifiedkfold


**Experiment - 2**

train xlmr base 5 kfolds



In [ ]:
%%time

#train_and_evaluate(trainStrategy="stratifiedkfold")


**Experiment - 3**

train xlm roberta base 8 stratified kfolds  

In [ ]:

%%time


#train_and_evaluate(trainStrategy="stratifiedkfold")


# References : 
1. https://www.kaggle.com/ragnar123/commonlit-readability-roberta-tf
2. https://www.kaggle.com/abhishek/step-1-create-folds
3. https://github.com/mhmoodlan/cyclic-learning-rate/blob/master/clr.py
4. https://www.kaggle.com/ajax0564/training-tf-roberta-on-mlm